# Paramètres <br/>
$D$ la demande du client <br/>
$PPA_t$ la production du PPA (supposée gratuite) <br/>
$p_t$ le prix de l'éléricité sur le marché <br/>
$e_{bat}$ l'éfficité de la batterie <br/>
$c_{bat}$ la capacité de la batterie <br/>
$f_{bat}$ le flux maximal de la batterie <br/>
$e_{elec}$ l'éfficité de l'électrolyseur <br/>
$c_{elec}$ la capacité de l'électrolyseur <br/>
$c_{tank}$ la capacité du réservoir <br/>

# Variables <br/>
$\text{charge}_t$ la charge de la batterie à l'heure $t$ (en MWh)<br/>
$\text{prod}_t$ la production de l'électrolyseur à l'heure $t$ (en kg $H_2$)<br/>
$\text{stock}_t$ le stock d'hydrogène dans le reservoir à l'heure $t$ (en kg $H_2$)<br/>
$\text{elec}_t$ la consommation d'élécricité venant de la grille à l'heure $t$ (en MWh) <br/>

# Contraintes <br/>

Satisfaction de la demande du client : <br/>
$\forall t \in [|1, T|], \text{prod}_t + \text{stock}_t = D_t + \text{stock}_{t+1}$ <br/>
Consomation d'élécricité : <br/>
$\forall t \in [|1, T|], \text{elec}_t + \text{PPA}_t + e_{bat} \times \text{charge}_t = e_{elec} \times \text{prod}_t + \text{charge}_{t+1}$ <br/>
Contraintes de capacité : <br/>
$\forall t \in [|1, T|], \text{charge}_t \leq c_{bat}$ <br/>
$\forall t \in [|1, T|], \text{prod}_t \times e_{elec} \leq c_{elec} $ <br/>
$\forall t \in [|1, T|], \text{stock}_t \leq c_{tank} $ <br/>
Contraintes de flux : <br/>
$\forall t \in [|1, T|], |\text{charge}_t - \text{charge}_{t+1}| \leq f_{bat} $ <br/>

# Objectif <br/>

Minimiser le coût de l'élécricité : <br/>
$\min \sum_{t=1}^{T} p_t \times \text{elec}_t$

In [1]:
from pulp import *

In [4]:
D = 1000
PPA = [50] * 18 + [40, 30, 20, 10, 0, 50]
Pt = [20.79, 17.41, 16.24, 11.9, 9.77, 15.88, 24.88, 29.7, 35.01, 33.95, 29.9, 29.03]
Pt += [27.07, 26.43, 27.53, 29.05, 31.42, 39.92, 41.3, 41.51, 39.75, 30.13, 30.36, 32.4]

ebat = 0.9


24